In [10]:
import tensorflow_federated as tff
tff.__version__

AttributeError: module 'tensorflow.python.util.tf_export' has no attribute 'ESTIMATOR_API_NAME'

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt 
from pycocotools.coco import COCO 
import os
import urllib.request

In [ ]:
# Download the annotations file for COCO dataset
# ANNOTATIONS_FILE = "instances_val2017.json"
# ANNOTATIONS_URL = f"http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
# if not os.path.exists(ANNOTATIONS_FILE):
#     urllib.request.urlretrieve(ANNOTATIONS_URL, "coco_annotations.zip")    !unzip -q coco_annotations.zip -d .

In [ ]:
# Load COCO dataset annotations
ANNOTATIONS_FILE = "instances_val2017.json"
coco = COCO(f'annotations/{ANNOTATIONS_FILE}')

# Function to load and preprocess images
def load_image(image_id):
    image_info = coco.loadImgs(image_id)[0]
    image_path = f"http://images.cocodataset.org/val2017/{image_info['file_name']}"
    
    image = tf.io.decode_jpeg(tf.io.read_file(tf.keras.utils.get_file(image_info['file_name'], image_path)))
    image = tf.image.resize(image, (128, 128))  # Resize for training
    image = image / 255.0  # Normalize pixels
    
    return image
# Get sample image IDs
image_ids = coco.getImgIds()[:10]  # Select 10 clients
# Create federated dataset
federated_train_data = [tf.data.Dataset.from_tensor_slices([load_image(img_id)]).batch(1) for img_id in image_ids]


loading annotations into memory...
Done (t=0.59s)
creating index...
index created!


In [ ]:
def create_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # Assuming 10 classes
    ])
    return model
# Convert to TFF model
def model_fn():
    keras_model = create_cnn_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )

TypeError: object of type 'TensorSpec' has no len()

In [ ]:
iterative_process = tff.learning.algorithms.build_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.001)
)
state = iterative_process.initialize()


TypeError: from_keras_model() got an unexpected keyword argument 'lengths'

In [ ]:
NUM_ROUNDS = 5
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

In [ ]:
evaluation_model = create_cnn_model()
state.model.assign_weights_to(evaluation_model)
# Load new test images
test_data = tf.data.Dataset.from_tensor_slices([load_image(img_id) for img_id in image_ids[5:10]]).batch(1)
loss, accuracy = evaluation_model.evaluate(test_data)
print(f'Test Accuracy: {accuracy:.4f}')